In [1]:
from glob import glob
import cv2
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input, Activation, Dense
import matplotlib.pyplot as plt
from scipy import spatial

C:\Users\Banipreet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X = {}
emotions = ['AN', 'SA', 'SU', 'HA', 'DI', 'FE', 'NE']
data = np.array(glob('dataset/JAFFE/*'))

In [3]:
for e in emotions:
    l = []
    for d in data:
        if e in d:
            l.append(d)
    X[e] = l

In [4]:
def AutoModel():
    inputs = Input(shape = (4096, ))
    encoder = Dense(2800)(inputs)
    encoder = Dense(500)(encoder)
    decoder = Dense(2800)(encoder)
    decoder = Dense(4096)(decoder)
    model = Model(input = inputs, output = decoder)
    return model

In [5]:
models = {}
for e in emotions:
    models[e] = AutoModel()

C:\Users\Banipreet\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
        shear_range=0.001,
        zoom_range=0.02,
        rotation_range=1,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
from sklearn.model_selection import train_test_split
X_test_files = {}
X_train_files = {}
for e in emotions:
    tr_files, te_files = train_test_split(X[e], test_size=0.25)
    X_test_files[e] = te_files
    X_train_files[e] = tr_files

In [8]:
from skimage import io,color
X_test = {}
X_train = {}
for e in emotions:
    l = []
    for img_path in X_train_files[e]:
        img = io.imread(img_path)
        img = cv2.resize(img,(64,64))
        if len(img.shape) == 3: img = color.rgb2gray(img)
        l.append(img)
    X_train[e] = np.array(l)
    X_train[e] = X_train[e].reshape(X_train[e].shape+(1,))
    l = []
    for img_path in X_test_files[e]:
        img = io.imread(img_path)
        img = cv2.resize(img,(64,64))
        if len(img.shape) == 3: img = color.rgb2gray(img)
        l.append(img)
    X_test[e] = np.array(l)
    X_test[e] = X_test[e].reshape(X_test[e].shape+(1,))

In [9]:
def generator(gen):
    for x in gen:
        x = x.reshape([x.shape[0],-1])
        yield (x,x)

In [10]:
#remove the quotes (uncomment) for training.
#use the already trained weights for the autoencoder
"""epochs = 10
cnt = 0
for e in emotions:
    if e!= 'NE': continue
    print('Autoencoder Emotion '+e)
    train_datagen.fit(X_train[e])
    test_datagen.fit(X_test[e])
    train_generator = train_datagen.flow(X_train[e])
    test_generator = test_datagen.flow(X_test[e])
    
    print(e+' emotion autoencoder')
    models[e].compile(optimizer='adam', loss='mse')
    models[e].fit_generator(generator(train_generator),
                            steps_per_epoch=100,
                            epochs=epochs,
                            validation_data=generator(test_generator),
                            validation_steps=20)
    models[e].save_weights('weights'+e+'4.h5')
"""
    

"epochs = 10\ncnt = 0\nfor e in emotions:\n    if e!= 'NE': continue\n    print('Autoencoder Emotion '+e)\n    train_datagen.fit(X_train[e])\n    test_datagen.fit(X_test[e])\n    train_generator = train_datagen.flow(X_train[e])\n    test_generator = test_datagen.flow(X_test[e])\n    \n    print(e+' emotion autoencoder')\n    models[e].compile(optimizer='adam', loss='mse')\n    models[e].fit_generator(generator(train_generator),\n                            steps_per_epoch=100,\n                            epochs=epochs,\n                            validation_data=generator(test_generator),\n                            validation_steps=20)\n    models[e].save_weights('weights'+e+'4.h5')\n"

In [11]:
def accuracy(dataset):
    correct = 0
    total = 0
    for e in emotions:
        for img in dataset[e]:
            total = total+1
            img = img.reshape([1,-1])
            img = img/255
            li = []
            for k in emotions:
                val = models[k].predict(img)
                li.append(1-spatial.distance.cosine(val, img))
            li = np.array(li)
            p = li.argmax()
            if emotions[p] == e: correct = correct+1
    return correct/total * 100

In [12]:
#reload the weights
for e in emotions:
    models[e].load_weights('weights'+e+'4.h5')

In [13]:
#Testing part
print('Train accuracy '+str(accuracy(X_train))+'%')
print('Test accuracy '+str(accuracy(X_test))+'%')        

Train accuracy 47.94520547945205%
Test accuracy 52.94117647058824%
